#Filtración colaborativa 



Para realizar Sistemas de Filtrado Colaborativo se requieren de algunas librerías, y una de ellas es pandas, la cual se especializa en el análisis y manejo de estructura de datos. La librería numpy permite realizar cálculo numérico y el análisis de datos, especialmente para un gran volumen de datos. Con cosine_similarity se puede calcular la similitud de coseno entre muestras en X e Y.  La función csr_matrix es utilizada para crear una matriz dispersa de formato de fila dispersa comprimida . La svds en cambio se refiere a la descomposición de valores singulares de una matriz dispersa. La librería train_test_split permite dividir un dataset en dos bloques, típicamente bloques destinados al entrenamiento y validación del modelo. Y otras librerías más que son de utilidad para realizar filtrado colaborativo 

In [ ]:
#cargar librerías
import pandas as pd
import numpy as np

#Permite calcular la similitud del coseno entre las muestras en X e Y.
from sklearn.metrics.pairwise import cosine_similarity

#Crear una matriz dispersa de formato de fila dispersa comprimida
from scipy.sparse import csr_matrix

#Descomposición parcial en valores singulares de una matriz dispersa.
from scipy.sparse.linalg import svds

#Dividir un dataset en dos bloques
from surprise.model_selection import train_test_split
from surprise import KNNWithMeans, Dataset, accuracy, Reader


Una vez realizados todas los import necesario, se declara una variable para que guarde el conjunto del data y cada vez que se realice una mención de la variable PATH se obtenga el "data.csv" 

In [ ]:
# constante para guardar los datos del Dataset
PATH = 'data.csv'

# Datos de importacion

Se realiza la lectura del dataset y con el “df.shape” se obtiene una tupla con el número de filas y columnas del DataFrame df

In [ ]:
#leer los datos del Dataset
df = pd.read_csv(PATH)
df.shape

(100000, 10)

In [ ]:
#Devuelve los primeros elementos de la estructura
df.head()

,book_id,author_id,book_genre,reader_id,num_pages,book_rating,publisher_id,publish_year,book_price,text_lang
0,655,52,4,11482,300,4,8,2012,94,7
1,2713,90,3,6479,469,1,8,2012,33,5
2,409,17,2,25472,435,1,12,2001,196,4
3,1150,234,10,23950,529,2,23,2019,79,2
4,2424,390,5,13046,395,2,20,2010,200,4


Para realizar el Filtrado colaborativo se puede clasificar en tres métodos, los cuales son: basados en elementos, usuarios e híbridas. A continuación se explica cada una de ellas.

# Método 1 - basado en elementos

Este método se recomienda en base a los basan en elementos como los ratings de ítems realizados por otros usuarios en el sistema. Y para realizar este método primero se genera una tabla dinámica con "reader_id" en el index, "book_id" en la columna y los valores vendrían siendo los "book_rating" que son las calificaciones. Después se crea una matriz dispersa con formato de fila dispersa comprimida con la función "csr_matriz"

In [ ]:
# Genere una tabla dinámica.
pt_df = df.pivot_table(
    columns = 'book_id',
    index = 'reader_id',
    values = 'book_rating'
).fillna(0)

# convertir a una matriz csr
mat = pt_df.values
mat = csr_matrix(mat)

Se establece una función en donde normaliza las entradas de "pred_reating"

In [ ]:
#Esta función normalizará la entrada pred_ratings
def normalize(pred_ratings):
    return (pred_ratings - pred_ratings.min()) / (pred_ratings.max() - pred_ratings.min())

La función generate_prediction_df realizará el cálculo de la descomposición de valor único de la matriz de entrada dados n_factores. Luego generará y normalizará las predicciones de calificación de los usuarios.

In [ ]:
def generate_prediction_df(mat, pt_df, n_factors):
    '''
    params:
        mat (CSR Matrix) : matriz scipy csr correspondiente a la tabla dinámica (pt_df)
        pt_df (DataFrame) : pandas dataframe que es una tabla dinámica
        n_factors (Integer) : Number of singular values and vectors to compute. 
                              Must be 1 <= n_factors < min(mat.shape). 
    '''
    if not 1 <= n_factors < min(mat.shape):
        raise ValueError("Must be 1 <= n_factors < min(mat.shape)")
        
    # factorización de matrices
    u, s, v = svds(mat, k = n_factors)
    s = np.diag(s)

    # calcular las calificaciones de pred
    pred_ratings = np.dot(np.dot(u, s), v) 
    pred_ratings = normalize(pred_ratings)
    
    # convertir a df
    pred_df = pd.DataFrame(
        pred_ratings,
        columns = pt_df.columns,
        index = list(pt_df.index)
    ).transpose()
    return pred_df

Se verifica el tiempo que toma para realizar la predición 

In [ ]:
%time pred_df = generate_prediction_df(mat, pt_df, 10)

CPU times: user 1.35 s, sys: 750 ms, total: 2.1 s
Wall time: 1.36 s


Con los parámetros de entrada de "usr_id" y "pred_df", la función realizará recomendaciones al usuario.

In [ ]:
def recommend_items(pred_df, usr_id, n_recs):
    '''
    params:
        pred_df (DataFrame) : generado a partir de la función `generate_prediction_df` 
        usr_id (Integer) : Se refiere al usuario para el que desea obtener recomendaciones de artículos
        n_recs (Integer) : El número de recomendaciones que desea para este usuario
    '''
    usr_pred = pred_df[usr_id].sort_values(ascending = False).reset_index().rename(columns = {usr_id : 'sim'})
    rec_df = usr_pred.sort_values(by = 'sim', ascending = False).head(n_recs)
    return rec_df

Se realiza las recomendaciones correctamente para usuario 5 con 4 recomendaciones solicitadas para el mismo.  

In [ ]:
recommend_items(pred_df, 5, 4)

,book_id,sim
0,1108,0.143244
1,1839,0.140945
2,725,0.140516
3,1897,0.140497


#Método 2 - basado en usuarios

En cambio, este método se recomienda en base a similitudes que encuentra entre el usuario actual con otro. Para realizar este método también se genera una tabla dinámica con "reader_id" en el index, "book_id" en la columna y los valores vendrían siendo los "book_rating" que son las calificaciones

In [ ]:
pt_df = df.pivot_table(
    columns = 'book_id', 
    index = 'author_id', 
    values = 'book_rating',
).fillna(0)


La función find_similar_readers() se encargará de encontrar los lectores similares al "lector_id" que se especifica por el usuario. 

In [ ]:
def find_similar_readers(pt_df, reader_id, n_recs):
    '''
    params:
        pred_df (DataFrame) : generado a partir de la función `generate_prediction_df` 
        usr_id (Integer) : Se refiere al usuario para el que desea obtener recomendaciones de artículos
        n_recs (Integer) : El número de recomendaciones que desea para este usuario
    '''
    
    # lector separado de interés y todos los demás lectores
    reader = pt_df[pt_df.index == reader_id]
    other_readers = pt_df[pt_df.index != reader_id]

    # obtener la similitud del lector actual y todos los demás lectores
    sim = cosine_similarity(reader, other_readers)[0].tolist()
    idx = other_readers.index.tolist()

    # crear un diccionario de similitud para este usuario con todos los demás usuarios
    idx_sim = dict(zip(idx, sim))
    idx_sim = sorted(idx_sim.items(), key=lambda x: x[1], reverse=True)

    similar_readers = idx_sim[:n_recs]
    readers = [rdr[0] for rdr in similar_readers]

    return readers

La función da como resultado que el usuario 226 tiene similitudes con los usuarios 319, 191, 145, 165 y 212. Se observa solo 5 usuarios porque solo se le solicitó que encontrará 5 y la función devuelve a los 5 primeros que analizó y encontró.

In [ ]:
find_similar_readers(pt_df = pt_df, reader_id = 226, n_recs = 5)

[319, 191, 145, 162, 212]

#Método 3 - híbrido

Como su nombre mismo lo dice, este método es una combinación del filtrado colaborativo basado en usuarios y elementos. Para realizar este método se declara una variable para que guarde el dataFrame con los elementos de "reader_id", "book_id", "book_rating "

In [ ]:
rdf = df[['reader_id', 'book_id', 'book_rating']]

Se realiza la lectura al dataset y se divide el conjunto de datos en dos partes, el primero es para realizar el entrenamiento y el segundo para realizar la pruebas. 

In [ ]:
#Lectura del conjunto de datos
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(rdf, reader)

In [ ]:
#Dividir el conjunto de datos
trainset, testset = train_test_split(data, test_size=0.3,random_state=10)


Para cambiar de filtrado colaborativo basado en usuario a basado en elemento se debe poner "True" en vez de "False" en user_based. 

In [ ]:
# Use verdadero/falso basado en usuario para cambiar entre filtrado colaborativo basado en usuario o basado en elementos
algo = KNNWithMeans(k=5, sim_options={'name': 'pearson_baseline', 'user_based': False})
algo.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [ ]:
# ejecutar el modelo entrenado contra el conjunto de pruebas
test_pred = algo.test(testset)

Tiene un error de 2.9306 entre la predición y el valor conocido. 

In [ ]:
# obtener RMSE
accuracy.rmse(test_pred, verbose=True)

RMSE: 2.9306


2.9306185721359865

In [ ]:
algo.predict(uid = 10, iid = 43)

Prediction(uid=10, iid=43, r_ui=None, est=5, details={'actual_k': 0, 'was_impossible': False})